In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/english-to-urdu-sentences-with-translation/English to Urdu.csv


In [51]:
import warnings
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
import string  # Add this import to use string.punctuation


In [52]:
df=pd.read_csv('/kaggle/input/english-to-urdu-sentences-with-translation/English to Urdu.csv')
df.head()

,SENTENCES,MEANING
0,How can I communicate with my parents?,میں اپنے والدین سے کیسے بات کروں ؟
1,How can I make friends?’,میں دوست کیسے بنائوں ؟
2,Why do I get so sad?’,میں اتنا اداس کیوں ہوں؟.
3,"If you’ve asked yourself such questions, you’r...",اگر آپ نے اپنے آپ سے ایسے سوالات کیے ہیں، تو آ...
4,"Depending on where you’ve turned for guidance,...",اس بات پر منحصر ہے کہ آپ رہنمائی کے لیے کہاں ...


In [53]:
df.columns

Index(['SENTENCES ', 'MEANING'], dtype='object')

In [54]:
df.rename(columns={'SENTENCES ': 'SENTENCES'}, inplace=True)

print(df.columns)

Index(['SENTENCES', 'MEANING'], dtype='object')


# 1.Data Preprocessing

In [55]:
df['SENTENCES'] = df['SENTENCES'].fillna("").astype(str)
df['MEANING'] = df['MEANING'].fillna("").astype(str)

>## Lowercasing and Removing Punctuation


In [56]:
df['SENTENCES'] = df['SENTENCES'].str.lower().str.translate(str.maketrans('', '', string.punctuation))
df['MEANING'] = df['MEANING'].str.translate(str.maketrans('', '', string.punctuation))


> ## TOKENIZATION

In [57]:
df['SENTENCES'] = df['SENTENCES'].apply(word_tokenize)
df['MEANING'] = df['MEANING'].apply(word_tokenize)

In [58]:
df

,SENTENCES,MEANING
0,"[how, can, i, communicate, with, my, parents]","[میں, اپنے, والدین, سے, کیسے, بات, کروں, ؟]"
1,"[how, can, i, make, friends, ’]","[میں, دوست, کیسے, بنائوں, ؟]"
2,"[why, do, i, get, so, sad, ’]","[میں, اتنا, اداس, کیوں, ہوں؟]"
3,"[if, you, ’, ve, asked, yourself, such, questi...","[اگر, آپ, نے, اپنے, آپ, سے, ایسے, سوالات, کیے,..."
4,"[depending, on, where, you, ’, ve, turned, for...","[اس, بات, پر, منحصر, ہے, کہ, آپ, رہنمائی, کے, ..."
...,...,...
30159,"[tasty, food, nice, environment, everyone, sho...","[لذیذ, کھانا, اچھا, ماحول, ہر, کسی, کو, دوستوں..."
30160,"[thumbs, up]","[بہت, خوب]"
30161,"[food, was, awesome]","[کھانا, لاجواب, تھا۔]"
30162,"[economical, place, with, a, great, taste, a, ...","[ایک, عظیم, ذائقہ, کے, ساتھ, اقتصادی, جگہ, کرا..."


> ## Split the datset into test train and cross validation set

In [59]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)


# 2.Vocabulary Creation

>## Build Vocabulary

In [60]:
from collections import Counter
english_vocab = {word: idx for idx, (word, _) in enumerate(Counter([w for s in train['SENTENCES'] for w in s]).most_common(10000), start=1)}
urdu_vocab = {word: idx for idx, (word, _) in enumerate(Counter([w for s in train['MEANING'] for w in s]).most_common(10000), start=1)}

# 3.Text To Sequence Coversion

In [61]:
def text_to_sequence(sentence, vocab):
    return [vocab.get(word, vocab.get('<unk>', 0)) for word in sentence]

train['SENTENCES'] = train['SENTENCES'].apply(lambda x: text_to_sequence(x, english_vocab))
train['MEANING'] = train['MEANING'].apply(lambda x: text_to_sequence(x, urdu_vocab))

In [62]:
train

,SENTENCES,MEANING
7132,"[37, 19, 4, 44, 4141, 9, 34, 1323]","[19, 25, 135, 1146, 3, 14, 109, 7]"
25770,"[32, 19, 3, 25, 9, 325, 6324, 105, 17, 4142, 2...","[1542, 6602, 3, 22, 25, 50, 3135, 47, 1, 34, 1..."
29908,"[35, 377, 377, 67, 802, 1594, 14, 228, 559, 4,...","[28, 1, 423, 423, 68, 8, 26, 101, 175, 6, 29, ..."
19449,"[29, 19, 10, 2109, 120, 6326]","[1730, 371, 4, 6603, 1, 16, 23, 25]"
3418,"[238, 25, 13, 112, 551]","[31, 494, 5, 316, 5278]"
...,...,...
29802,"[46, 26, 47, 4000, 12, 6199, 459, 406, 83, 400...","[52, 1, 82, 51, 27, 48, 123, 3, 492, 1, 370, 6..."
5390,"[20, 134, 38, 0, 97, 98, 1400, 147, 490, 148]","[21, 13, 4, 3369, 10, 22, 79, 396, 39, 21, 124..."
860,"[13, 351, 98, 708, 8, 1824, 3, 0]","[31, 701, 1, 16, 2394, 560, 146, 39]"
15795,"[2, 1, 8825, 667, 27, 148, 6, 550, 3393]","[2, 60, 5, 124, 5263, 10, 38, 1005, 4, 4611, 5..."


In [63]:
print(train.head())
print(train['SENTENCES'].iloc[0])
print(train['MEANING'].iloc[0])


                                               SENTENCES  \
7132                  [37, 19, 4, 44, 4141, 9, 34, 1323]   
25770  [32, 19, 3, 25, 9, 325, 6324, 105, 17, 4142, 2...   
29908  [35, 377, 377, 67, 802, 1594, 14, 228, 559, 4,...   
19449                      [29, 19, 10, 2109, 120, 6326]   
3418                             [238, 25, 13, 112, 551]   

                                                 MEANING  
7132                  [19, 25, 135, 1146, 3, 14, 109, 7]  
25770  [1542, 6602, 3, 22, 25, 50, 3135, 47, 1, 34, 1...  
29908  [28, 1, 423, 423, 68, 8, 26, 101, 175, 6, 29, ...  
19449                [1730, 371, 4, 6603, 1, 16, 23, 25]  
3418                             [31, 494, 5, 316, 5278]  
[37, 19, 4, 44, 4141, 9, 34, 1323]
[19, 25, 135, 1146, 3, 14, 109, 7]


In [64]:
import pandas as pd

# Replace 'df' with your dataset variable
def clean_data(df):
    # Ensure columns are correctly named
    df.rename(columns={'SENTENCES ': 'SENTENCES', 'MEANING ': 'MEANING'}, inplace=True)

    # Replace 'nisc' in lists
    def clean_column(column):
        return column.apply(lambda x: [word for word in x if word != 'nisc'])

    df['SENTENCES'] = clean_column(df['SENTENCES'])
    df['MEANING'] = clean_column(df['MEANING'])

    # Remove rows where SENTENCES or MEANING are empty lists
    df = df[df['SENTENCES'].apply(bool) & df['MEANING'].apply(bool)]
    
    return df

# Apply the cleaning function
df = clean_data(df)

# Validate the cleaning process
print(df.head())


                                           SENTENCES  \
0      [how, can, i, communicate, with, my, parents]   
1                    [how, can, i, make, friends, ’]   
2                      [why, do, i, get, so, sad, ’]   
3  [if, you, ’, ve, asked, yourself, such, questi...   
4  [depending, on, where, you, ’, ve, turned, for...   

                                             MEANING  
0        [میں, اپنے, والدین, سے, کیسے, بات, کروں, ؟]  
1                       [میں, دوست, کیسے, بنائوں, ؟]  
2                      [میں, اتنا, اداس, کیوں, ہوں؟]  
3  [اگر, آپ, نے, اپنے, آپ, سے, ایسے, سوالات, کیے,...  
4  [اس, بات, پر, منحصر, ہے, کہ, آپ, رہنمائی, کے, ...  


# 4.Padding Sequences
>Pad sequences to ensure they have the same length for batch processing

In [66]:
max_len_source = max(train['SENTENCES'].apply(len))
max_len_target = max(train['MEANING'].apply(len))


train['SENTENCES'] = pad_sequences(train['SENTENCES'], maxlen=max_len_source, padding='post').tolist()
train['MEANING'] = pad_sequences(train['MEANING'], maxlen=max_len_target, padding='post').tolist()


In [67]:
print(train['SENTENCES'].apply(len).unique())
print(train['MEANING'].apply(len).unique())


[436]
[923]


In [68]:
print(train['SENTENCES'].head())
print(train['MEANING'].head())


7132     [37, 19, 4, 44, 4141, 9, 34, 1323, 0, 0, 0, 0,...
25770    [32, 19, 3, 25, 9, 325, 6324, 105, 17, 4142, 2...
29908    [35, 377, 377, 67, 802, 1594, 14, 228, 559, 4,...
19449    [29, 19, 10, 2109, 120, 6326, 0, 0, 0, 0, 0, 0...
3418     [238, 25, 13, 112, 551, 0, 0, 0, 0, 0, 0, 0, 0...
Name: SENTENCES, dtype: object
7132     [19, 25, 135, 1146, 3, 14, 109, 7, 0, 0, 0, 0,...
25770    [1542, 6602, 3, 22, 25, 50, 3135, 47, 1, 34, 1...
29908    [28, 1, 423, 423, 68, 8, 26, 101, 175, 6, 29, ...
19449    [1730, 371, 4, 6603, 1, 16, 23, 25, 0, 0, 0, 0...
3418     [31, 494, 5, 316, 5278, 0, 0, 0, 0, 0, 0, 0, 0...
Name: MEANING, dtype: object


# 5.Creating The Model

>## Encoder and Decoder

In [77]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Reload original train data if necessary
train = df.sample(frac=0.8, random_state=42)  # Example split, replace with your actual split logic

# Fit tokenizers on original text
tokenizer_source = Tokenizer(num_words=10000, oov_token="<unk>")
tokenizer_target = Tokenizer(num_words=10000, oov_token="<unk>")

tokenizer_source.fit_on_texts(train['SENTENCES'])
tokenizer_target.fit_on_texts(train['MEANING'])

# Get vocabulary sizes
vocab_size_source = len(tokenizer_source.word_index) + 1  # Add 1 for padding token
vocab_size_target = len(tokenizer_target.word_index) + 1

# Calculate maximum sequence lengths
max_encoder_seq_length = max(train['SENTENCES'].apply(len))
max_decoder_seq_length = max(train['MEANING'].apply(len))


# Convert text to sequences
encoder_input_data = pad_sequences(tokenizer_source.texts_to_sequences(train['SENTENCES']), 
                                    maxlen=max_encoder_seq_length, padding='post')

decoder_input_data = pad_sequences(tokenizer_target.texts_to_sequences(train['MEANING'].apply(lambda x: x[:-1])), 
                                    maxlen=max_decoder_seq_length, padding='post')

decoder_target_data = pad_sequences(tokenizer_target.texts_to_sequences(train['MEANING'].apply(lambda x: x[1:])), 
                                     maxlen=max_decoder_seq_length, padding='post')

# Model parameters
embedding_dim = 256
latent_dim = 512
vocab_size_source = len(english_vocab) + 1
vocab_size_target = len(urdu_vocab) + 1

# Encoder
encoder_inputs = Input(shape=(None,), name="encoder_inputs")
encoder_embedding = Embedding(vocab_size_source, embedding_dim, mask_zero=True, name="encoder_embedding")(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True, name="encoder_lstm")
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,), name="decoder_inputs")
decoder_embedding = Embedding(vocab_size_target, embedding_dim, mask_zero=True, name="decoder_embedding")(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_target, activation="softmax", name="decoder_dense")
decoder_outputs = decoder_dense(decoder_outputs)

In [78]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [83]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow as tf

# Set an initial learning rate
initial_learning_rate = 0.001

# Set the decay parameters
lr_schedule = ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=100000,  # Number of steps before learning rate decays
    decay_rate=0.96,     # Factor by which the learning rate is reduced
    staircase=True       # If True, the learning rate decays in discrete intervals
)

# Compile the model with the learning rate schedule and sparse categorical crossentropy loss
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='sparse_categorical_crossentropy',  # Correct for softmax output and integer labels
              metrics=['accuracy'])


# Train the Model

In [84]:
history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=32,
    epochs=30,
    validation_split=0.2
)

Epoch 1/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 411s 690ms/step - accuracy: 9.3895e-04 - loss: 6.6646 - val_accuracy: 0.0022 - val_loss: 5.3249
Epoch 2/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 405s 684ms/step - accuracy: 0.0025 - loss: 5.1167 - val_accuracy: 0.0028 - val_loss: 4.8617
Epoch 3/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 406s 684ms/step - accuracy: 0.0032 - loss: 4.5601 - val_accuracy: 0.0032 - val_loss: 4.6291
Epoch 4/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 406s 685ms/step - accuracy: 0.0037 - loss: 4.1739 - val_accuracy: 0.0035 - val_loss: 4.4668
Epoch 5/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 406s 685ms/step - accuracy: 0.0041 - loss: 3.8431 - val_accuracy: 0.0038 - val_loss: 4.3436
Epoch 6/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 406s 686ms/step - accuracy: 0.0046 - loss: 3.5187 - val_accuracy: 0.0041 - val_loss: 4.2538
Epoch 7/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 407s 686ms/step - accuracy: 0.0052 - loss: 3.2223 - val_accuracy: 0.0043 - val_loss: 4.2033
Epoch 8/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 407s 687ms/step - accuracy: 0.00

In [ ]:
model.summary()

In [86]:
model.save('English to Urdu translator(NMT Model).h5')
